In [28]:
indices1 = [0]

cur = 0
for i in range(0, 20):
    if (i + 1) % 5 == 0:
        cur += 459_760
    else:
        cur += 1_000_000
    indices1.append(cur)

print(indices1)
# Create 2-tuples of the form (start, end)
ranges1 = [(indices1[i], indices1[i + 1], indices1[i + 1] - indices1[i]) for i in range(0, 20)]

indices2 = [0]

cur = 0
for i in range(0, 20):
    # if i == 4 or i == 9:
    #     cur += 459_759
    if (i + 1) % 5 == 0:
        cur += 459_759
    else:
        cur += 1_000_000
    indices2.append(cur)

print(indices2)
# Create 2-tuples of the form (start, end)
ranges2 = [(indices2[i], indices2[i + 1], indices2[i + 1] - indices2[i]) for i in range(0, 20)]

[0, 1000000, 2000000, 3000000, 4000000, 4459760, 5459760, 6459760, 7459760, 8459760, 8919520, 9919520, 10919520, 11919520, 12919520, 13379280, 14379280, 15379280, 16379280, 17379280, 17839040]
[0, 1000000, 2000000, 3000000, 4000000, 4459759, 5459759, 6459759, 7459759, 8459759, 8919518, 9919518, 10919518, 11919518, 12919518, 13379277, 14379277, 15379277, 16379277, 17379277, 17839036]


In [29]:
import torch

In [31]:
split_embeddings = [
    torch.zeros((indices1[-1] + 1, 1024), dtype=torch.float16, device='cuda:0'),
    torch.zeros((indices2[-1], 1024), dtype=torch.float16, device='cuda:1')
]

In [30]:
del split_embeddings
torch.cuda.empty_cache()

In [15]:
shard_paths = [
    f'/burg/dsi/users/jef2182/data/e5-large-index/e5-large-shard-{i}.pt' for i in range(0, 40)
]

In [ ]:
for i in range(20):
    print(f'Loading shard {i}')
    print(f'{(ranges1[i][0], ranges1[i][1])}')
    split_embeddings[0][ranges1[i][0]:ranges1[i][1]] = torch.load(shard_paths[i], map_location=torch.device("cuda:0"), weights_only=True)

Loading shard 0
(0, 1000000)
Loading shard 1
(1000000, 2000000)
Loading shard 2
(2000000, 3000000)
Loading shard 3
(3000000, 4000000)
Loading shard 4
(4000000, 4459760)
Loading shard 5
(4459760, 5459760)
Loading shard 6
(5459760, 6459760)
Loading shard 7
(6459760, 7459760)
Loading shard 8
(7459760, 8459760)
Loading shard 9
(8459760, 8919520)
Loading shard 10
(8919520, 9919520)
Loading shard 11
(9919520, 10919520)
Loading shard 12
(10919520, 11919520)
Loading shard 13
(11919520, 12919520)
Loading shard 14
(12919520, 13379280)
Loading shard 15
(13379280, 14379280)
Loading shard 16
(14379280, 15379280)
Loading shard 17
(15379280, 16379280)
Loading shard 18
(16379280, 17379280)
Loading shard 19
(17379280, 17839040)


In [32]:
for i in range(20,39):
    print(f'Loading shard {i}')
    shard = torch.load(shard_paths[i], map_location=torch.device("cuda:0"), weights_only=True)
    print(shard.shape)
    split_embeddings[1][ranges2[i-20][0]:ranges2[i-20][1]] = shard

Loading shard 20
torch.Size([1000000, 1024])
Loading shard 21
torch.Size([1000000, 1024])
Loading shard 22
torch.Size([1000000, 1024])
Loading shard 23
torch.Size([1000000, 1024])
Loading shard 24
torch.Size([459759, 1024])
Loading shard 25
torch.Size([1000000, 1024])
Loading shard 26
torch.Size([1000000, 1024])
Loading shard 27
torch.Size([1000000, 1024])
Loading shard 28
torch.Size([1000000, 1024])
Loading shard 29
torch.Size([459759, 1024])
Loading shard 30
torch.Size([1000000, 1024])
Loading shard 31
torch.Size([1000000, 1024])
Loading shard 32
torch.Size([1000000, 1024])
Loading shard 33
torch.Size([1000000, 1024])
Loading shard 34
torch.Size([459759, 1024])
Loading shard 35
torch.Size([1000000, 1024])
Loading shard 36
torch.Size([1000000, 1024])
Loading shard 37
torch.Size([1000000, 1024])
Loading shard 38
torch.Size([1000000, 1024])


In [6]:
torch.load('/burg/dsi/users/jef2182/data/e5-large-index/e5-large-shard-38.pt', weights_only=True, map_location=torch.device('cuda:0')).shape

torch.Size([1000000, 1024])

In [7]:
torch.load('/burg/dsi/users/jef2182/data/e5-large-index/e5-large-shard-4.pt', weights_only=True, map_location=torch.device('cuda:0')).shape

torch.Size([459760, 1024])

In [9]:
torch.load('/burg/dsi/users/jef2182/data/e5-large-index/e5-large-shard-19.pt', weights_only=True, map_location=torch.device('cuda:0')).shape

torch.Size([459760, 1024])